In [1]:
import requests
from langchain import agents
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

from openinference.instrumentation.langchain import LangChainInstrumentor


In [ ]:

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
tracer_provider.add_span_processor(SimpleSpanProcessor(ConsoleSpanExporter()))

LangChainInstrumentor().instrument(tracer_provider=tracer_provider)


In [ ]:


@tool
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    return requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    ).json()



In [2]:
from langchain_ollama import ChatOllama


tools = [get_exchange_rate]
#llm = ChatOpenAI()
llm = ChatOllama(model="llama3.2:latest", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)




> Entering new AgentExecutor chain...
{
    "name": "RunnableLambda",
    "context": {
        "trace_id": "0xc38e9a89728949df3b9831ff1a9a7c32",
        "span_id": "0xb57d4942bc3ac9db",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x10fb4716047d7ed1",
    "start_time": "2025-02-05T13:30:38.089707Z",
    "end_time": "2025-02-05T13:30:38.090707Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "input.value": "{\"input\": \"What is the exchange rate from US dollars to Swedish currency today?\", \"intermediate_steps\": []}",
        "input.mime_type": "application/json",
        "output.value": "{\"output\": []}",
        "output.mime_type": "application/json",
        "openinference.span.kind": "CHAIN"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "telemetry.sdk.language": "python",
            "telemetry.sdk.name": "opentelemetry",
            "telemetry.sdk.versio

In [ ]:
agent = agents.create_tool_calling_agent(llm, tools, prompt)
agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)

if __name__ == "__main__":
    agent_executor.invoke(
        {"input": "What is the exchange rate from US dollars to Swedish currency today?"}
    )